In [107]:
import wfdb
import numpy as np
import pandas as pd

from glob import glob
import argparse
import os

data_dir = 'data\CPSC_2018\main-data'
reference_csv = os.path.join(data_dir, 'reference.csv')
label_csv = os.path.join(data_dir, 'labels.csv')

In [108]:
#handle split_path in Windows or Linux
split_type = os.path.sep
print(split_type)
#1. change split type to '/' (Windows)

def gen_reference_csv(data_dir, reference_csv):
    if not os.path.exists(reference_csv):
        recordpaths = glob(os.path.join(data_dir, '*.hea'))
        results = []
        for recordpath in recordpaths:
            patient_id = recordpath.split(split_type)[-1][:-4]
            # print(patient_id)
            # print(recordpath)
            # print(recordpath[:-4])
            _, meta_data = wfdb.rdsamp(recordpath[:-4])
            sample_rate = meta_data['fs']
            signal_len = meta_data['sig_len']
            age = meta_data['comments'][0]
            sex = meta_data['comments'][1]
            dx = meta_data['comments'][2]
            age = age[5:] if age.startswith('Age: ') else np.NaN
            sex = sex[5:] if sex.startswith('Sex: ') else 'Unknown'
            dx = dx[4:] if dx.startswith('Dx: ') else ''
            print(age,sex,dx)
            results.append([patient_id, sample_rate, signal_len, age, sex, dx])
        df = pd.DataFrame(data=results, columns=['patient_id', 'sample_rate', 'signal_len', 'age', 'sex', 'dx'])
        df.sort_values('patient_id').to_csv(reference_csv, index=None)




\


In [109]:
def gen_label_csv(label_csv, reference_csv, dx_dict, classes):
    if not os.path.exists(label_csv):
        results = []
        df_reference = pd.read_csv(reference_csv)
        for _, row in df_reference.iterrows():
            patient_id = row['patient_id']
            dxs = [dx_dict.get(code, '') for code in row['dx'].split(',')]
            labels = [0] * 9
            for idx, label in enumerate(classes):
                if label in dxs:
                    labels[idx] = 1
            results.append([patient_id] + labels)
        df = pd.DataFrame(data=results, columns=['patient_id'] + classes)
        n = len(df)
        folds = np.zeros(n, dtype=np.int8)
        for i in range(10):
            start = int(n * i / 10)
            end = int(n * (i + 1) / 10)
            folds[start:end] = i + 1
        df['fold'] = np.random.permutation(folds)
        columns = df.columns
        df['keep'] = df[classes].sum(axis=1)
        df = df[df['keep'] > 0]
        df[columns].to_csv(label_csv, index=None)

In [110]:
# if __name__ == "__main__":
leads = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
dx_dict = {
    '426783006': 'SNR', # Normal sinus rhythm
    '164889003': 'AF', # Atrial fibrillation
    '270492004': 'IAVB', # First-degree atrioventricular block
    '164909002': 'LBBB', # Left bundle branch block
    '713427006': 'RBBB', # Complete right bundle branch block
    '59118001': 'RBBB', # Right bundle branch block
    '284470004': 'PAC', # Premature atrial contraction
    '63593006': 'PAC', # Supraventricular premature beats
    '164884008': 'PVC', # Ventricular ectopics
    '429622005': 'STD', # ST-segment depression
    '164931005': 'STE', # ST-segment elevation
}
classes = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']
# parser = argparse.ArgumentParser()
# parser.add_argument('--data-dir', type=str, default='data/CPSC', help='Directory to dataset')
# args = parser.parse_args()



In [112]:
gen_reference_csv(data_dir, reference_csv)

74 Male 59118001
49 Female 426783006
81 Female 164889003
45 Male 164889003
53 Male 164884008
29 Male 59118001
74 Male 164889003
80 Female 429622005
81 Male 164889003
82 Male 59118001
87 Male 164909002
74 Male 164884008
57 Female 429622005
76 Female 429622005
41 Female 59118001
14 Female 426783006
60 Female 164889003
92 Female 164909002
74 Female 164889003
27 Female 426783006
64 Male 164931005
27 Female 59118001
67 Female 164889003
64 Female 164884008
74 Female 164884008
74 Male 164889003
55 Female 59118001
58 Female 59118001
35 Female 426783006
46 Male 426783006
62 Female 164884008
30 Male 164931005
34 Female 429622005
61 Male 164931005
52 Male 59118001
62 Male 164931005
42 Female 426783006
55 Male 426783006
17 Male 270492004
28 Female 429622005
40 Female 426783006
88 Male 270492004
85 Female 164889003,59118001
84 Male 164909002
67 Male 164884008
31 Female 284470004
54 Female 429622005
85 Female 284470004
67 Male 164884008


In [113]:
gen_label_csv(label_csv, reference_csv, dx_dict, classes)

In [114]:
#reference
if not os.path.exists(reference_csv):
        recordpaths = glob(os.path.join(data_dir, '*.hea'))
        results = []
        for recordpath in recordpaths:
            patient_id = recordpath.split(split_type)[-1][:-4]
            # print(patient_id)
            # print(recordpath)
            # print(recordpath[:-4])
            _, meta_data = wfdb.rdsamp(recordpath[:-4])
            sample_rate = meta_data['fs']
            signal_len = meta_data['sig_len']
            age = meta_data['comments'][0]
            sex = meta_data['comments'][1]
            dx = meta_data['comments'][2]
            age = age[5:] if age.startswith('Age: ') else np.NaN
            sex = sex[5:] if sex.startswith('Sex: ') else 'Unknown'
            dx = dx[4:] if dx.startswith('Dx: ') else ''
            print(age,sex,dx)
            results.append([patient_id, sample_rate, signal_len, age, sex, dx])
        df = pd.DataFrame(data=results, columns=['patient_id', 'sample_rate', 'signal_len', 'age', 'sex', 'dx'])
        df.sort_values('patient_id').to_csv(reference_csv, index=None)


In [116]:
#label
if not os.path.exists(label_csv):
        results = []
        df_reference = pd.read_csv(reference_csv)
        for _, row in df_reference.iterrows():
            patient_id = row['patient_id']
            dxs = [dx_dict.get(code, '') for code in row['dx'].split(',')]
            labels = [0] * 9
            for idx, label in enumerate(classes):
                if label in dxs:
                    labels[idx] = 1
            results.append([patient_id] + labels)
        df = pd.DataFrame(data=results, columns=['patient_id'] + classes)
        n = len(df)
        folds = np.zeros(n, dtype=np.int8)
        for i in range(10):
            start = int(n * i / 10)
            end = int(n * (i + 1) / 10)
            folds[start:end] = i + 1
        df['fold'] = np.random.permutation(folds)
        columns = df.columns
        df['keep'] = df[classes].sum(axis=1)
        df = df[df['keep'] > 0]
        df[columns].to_csv(label_csv, index=None)